In [1]:
from dataloader import DataLoader
import pandas as pd

files = ["20170" + str(i) + "-citibike-tripdata.csv" if i < 10 else "2017" + str(i) + "-citibike-tripdata.csv"
        for i in range(1,13)]

         
dl = DataLoader("../data",files)


dl.load_data()

100%|██████████| 12/12 [02:32<00:00, 12.72s/it]


In [86]:
stations = dl.load_stations()

In [87]:
dl.data.shape

(726676, 15)

In [7]:
group = dl.data.groupby(['start station id'])
stats_start = group.agg({
    'trip duration': ['median','mean']
})

stats_start['count'] = group.size()
stats_start = stats_start.reset_index()
stats_start.columns = ['id','median','mean','count']

group_end = dl.data.groupby(['end station id'])
stats_end = group_end.agg({
    'trip duration': ['median','mean']
})

stats_end['count'] = group_end.size()
stats_end = stats_end.reset_index()
stats_end.columns = ['id','median','mean','count']

station_stats = stats_end.merge(stats_start,on = 'id',suffixes = ['_start','_end'])

from sklearn.preprocessing import StandardScaler
X = station_stats.copy()
del X['id']
X = StandardScaler().fit_transform(X)
X[0]

array([-0.03647522, -0.04859765,  0.83196467,  0.04393115, -0.09977938,
        0.83956237])

# OPTICS

In [9]:
from sklearn.cluster import OPTICS, cluster_optics_dbscan
clust = OPTICS(min_samples=180, xi=.05, min_cluster_size=.4)

# Run the fit
clust.fit(X)

labels_050 = cluster_optics_dbscan(reachability=clust.reachability_,
                                   core_distances=clust.core_distances_,
                                   ordering=clust.ordering_, eps=0.3)
labels_200 = cluster_optics_dbscan(reachability=clust.reachability_,
                                   core_distances=clust.core_distances_,
                                   ordering=clust.ordering_, eps=2)

y_pred = labels_050
station_stats['label'] = y_pred
station_2017 = stations.merge(station_stats,left_on = 'station id', right_on='id')

In [28]:
station_stats['diffrence_count'] = station_stats['count_start'] - station_stats['count_end']
station_stats.groupby(['label']).agg({
    'diffrence_count' : 'mean'
})

,diffrence_count
label,
-1,16.035917
0,-31.202847


## DBScan

In [22]:
from sklearn.cluster import DBSCAN
print(X.shape)
db = DBSCAN(eps=0.30, min_samples=200,metric = 'l2').fit(X)
y_pred = db.labels_
station_stats['label'] = y_pred
station_2017 = stations.merge(station_stats,left_on = 'station id', right_on='id')

(810, 6)


In [23]:
station_stats['diffrence_count'] = station_stats['count_start'] - station_stats['count_end']
station_stats.groupby(['label']).agg({
    'diffrence_count' : 'mean'
})

,diffrence_count
label,
-1,16.035917
0,-31.202847


# Mapa

In [24]:
import plotly.graph_objects as go

spotify_green = '#1DB954'
token = 'pk.eyJ1IjoiZGFuaWVscG9uaWtvd3NraSIsImEiOiJjazRjempwb3owZ2N3M2xtMHBtZjZ6dGZ6In0.r99JQYbCq6Kevwj6DsWtGg'

fig = go.Figure(go.Scattermapbox(
        lon = station_2017['station longitude'],
        lat = station_2017['station latitude'],
        text = station_2017['label'],    
        mode='markers',
        marker=go.scattermapbox.Marker(
            size = 9,
            color = station_2017.label,
            opacity = 0.7
#             opacity = station_20190101_16['count']/station_20190101_16['count'].max()
#             colorscale= 'brwnyl',
#             showscale = True # jak sie ustali skale to pokazuje skale z boku
#             reversescale = True # odwraca skale kolorow
        )
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken= token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat= stations.loc[0,'station latitude'],
            lon= stations.loc[0,'station longitude']
        ),
        pitch=0,
        zoom=11,
        style="dark"
    )
)

fig.show()

In [12]:
station_stats.sort_values('count_start',ascending=False)

,id,median_start,mean_start,count_start,median_end,mean_end,count_end
273,519,604.0,7.755462e+02,160387,615.0,834.145142,162716
254,497,520.0,7.085308e+02,117318,528.0,777.880153,112218
170,402,538.0,7.053180e+02,116509,538.0,758.266424,108590
190,426,983.0,1.121731e+03,114511,925.0,1126.753982,105610
197,435,493.0,6.603391e+02,108457,500.0,727.923077,107133
...,...,...,...,...,...,...,...
629,3446,2430.0,2.892000e+03,4,185.5,185.500000,2
463,3248,317.0,3.550000e+02,3,714.5,714.500000,2
793,3631,4670242.0,4.670242e+06,2,293.0,293.000000,1
713,3543,1088.5,1.088500e+03,2,434.0,434.000000,1


In [15]:
y = stations.merge(station_stats,right_on=['id'],left_on=['station id'])

In [16]:
y

,station id,station name,station latitude,station longitude,id,median_start,mean_start,count_start,median_end,mean_end,count_end
0,3226,W 82 St & Central Park West,40.782750,-73.971370,3226,825.0,1.160859e+03,20870,756.0,1081.690003,22897
1,3263,Cooper Square & E 7 St,40.729236,-73.990868,3263,428.0,6.343296e+02,76580,466.0,687.611632,76219
2,3263,Cooper Square & Astor Pl,40.729515,-73.990753,3263,428.0,6.343296e+02,76580,466.0,687.611632,76219
3,3143,5 Ave & E 78 St,40.776829,-73.963888,3143,1190.0,1.468198e+03,33339,1099.0,1458.468522,34214
4,3143,5 Ave & E 78 St,40.776321,-73.964274,3143,1190.0,1.468198e+03,33339,1099.0,1458.468522,34214
...,...,...,...,...,...,...,...,...,...,...,...
824,3650,8D Mobile 01,45.505851,-73.569109,3650,879.0,2.733154e+03,65,892.5,2825.787879,66
825,3654,31 St & Northern Blvd,40.751870,-73.933493,3654,696.0,8.792308e+02,26,619.0,760.260870,23
826,3653,31 St & 35 Ave,40.758205,-73.928322,3653,301.5,5.290625e+02,32,340.0,476.120000,25
827,3631,Crown St & Bedford Ave,40.666563,-73.956741,3631,4670242.0,4.670242e+06,2,293.0,293.000000,1


In [21]:
import plotly.graph_objects as go

spotify_green = '#1DB954'
token = 'pk.eyJ1IjoiZGFuaWVscG9uaWtvd3NraSIsImEiOiJjazRjempwb3owZ2N3M2xtMHBtZjZ6dGZ6In0.r99JQYbCq6Kevwj6DsWtGg'

fig = go.Figure(go.Scattermapbox(
        lon = y['station longitude'],
        lat = y['station latitude'],
#         text = y['label'],    
        mode='markers',
        marker=go.scattermapbox.Marker(
            size = 9,
            color = y['count_end'] - y['count_start'],
            opacity = 0.7,
#             opacity = station_20190101_16['count']/station_20190101_16['count'].max()
            colorscale = 'brwnyl',
            showscale = True # jak sie ustali skale to pokazuje skale z boku
#             reversescale = True # odwraca skale kolorow
        )
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken= token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat= stations.loc[0,'station latitude'],
            lon= stations.loc[0,'station longitude']
        ),
        pitch=0,
        zoom=11,
        style="dark"
    )
)

fig.show()

## inaczej

In [25]:
X = dl.data.loc[(dl.data['trip duration'] > 120) & (
    dl.data['start station latitude'] != 0),
                ['start station latitude','start station longitude']].sample(700000)

X.shape

(700000, 2)

In [26]:
from sklearn.cluster import KMeans

clust = KMeans(n_clusters=10)
clust.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [27]:
clust.cluster_centers_

array([[ 40.79152333, -73.96183948],
       [ 40.72561509, -73.98803586],
       [ 40.74140367, -73.99867167],
       [ 40.68218862, -73.9686081 ],
       [ 40.74890017, -73.9813738 ],
       [ 40.71460703, -74.00813735],
       [ 40.76719736, -73.98525381],
       [ 40.76519751, -73.95830572],
       [ 40.68902493, -73.99229369],
       [ 40.71655401, -73.95439754]])

In [28]:
to_draw = pd.DataFrame({"lat":X.iloc[:,0],
                       "lon":X.iloc[:,1],
                       "label":clust.labels_}).drop_duplicates()

import plotly.graph_objects as go

spotify_green = '#1DB954'
token = 'pk.eyJ1IjoiZGFuaWVscG9uaWtvd3NraSIsImEiOiJjazRjempwb3owZ2N3M2xtMHBtZjZ6dGZ6In0.r99JQYbCq6Kevwj6DsWtGg'

fig = go.Figure(go.Scattermapbox(
        lat = to_draw['lat'],
        lon = to_draw['lon'],
        text = to_draw['label'],    
        mode='markers',
        marker=go.scattermapbox.Marker(
            size = 9,
            color = to_draw['label'],
            opacity = 0.7,
#             opacity = station_20190101_16['count']/station_20190101_16['count'].max()
#             colorscale = 'brwnyl',
#             showscale = True # jak sie ustali skale to pokazuje skale z boku
#             reversescale = True # odwraca skale kolorow
        )
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken= token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat= clust.cluster_centers_[0,0],
            lon= clust.cluster_centers_[0,1]
        ),
        pitch=0,
        zoom=11,
        style="dark"
    )
)

fig.show()

In [29]:
border_cluster = {
    0:[6,7],
    1:[5,2,4,9,8],
    2:[6,4,1,5],
    3:[8,9],
    4:[7,9,1,2,6],
    5:[2,1,8],
    6:[0,7,4,2],
    7:[0,6,4,9],
    8:[5,1,3],
    9:[7,4,1,8,3]
}

In [30]:
station_label = to_draw.merge(dl.data,
              left_on=['lat','lon'],right_on=['start station latitude','start station longitude'],
             how = "inner").merge(to_draw,
              right_on=['lat','lon'],left_on=['end station latitude','end station longitude'],
             how = "inner")

station_label.head()

,lat_x,lon_x,label_x,trip duration,start time,stop time,start station id,start station name,start station latitude,start station longitude,...,end station name,end station latitude,end station longitude,bike id,user type,birth year,gender,lat_y,lon_y,label_y
0,40.752062,-73.981632,4,1342,2017-01-01 00:12:16,2017-01-01 00:34:38,153,E 40 St & 5 Ave,40.752062,-73.981632,...,E 3 St & 1 Ave,40.724677,-73.987834,25023,Subscriber,1968.0,2,40.724677,-73.987834,1
1,40.752062,-73.981632,4,1032,2017-01-25 18:06:54,2017-01-25 18:24:07,153,E 40 St & 5 Ave,40.752062,-73.981632,...,E 3 St & 1 Ave,40.724677,-73.987834,18893,Subscriber,1986.0,1,40.724677,-73.987834,1
2,40.752062,-73.981632,4,2700,2017-02-18 17:08:11,2017-02-18 17:53:12,153,E 40 St & 5 Ave,40.752062,-73.981632,...,E 3 St & 1 Ave,40.724677,-73.987834,26389,Subscriber,1984.0,1,40.724677,-73.987834,1
3,40.752062,-73.981632,4,2141,2017-02-23 22:56:04,2017-02-23 23:31:46,153,E 40 St & 5 Ave,40.752062,-73.981632,...,E 3 St & 1 Ave,40.724677,-73.987834,16775,Subscriber,1994.0,1,40.724677,-73.987834,1
4,40.752062,-73.981632,4,1482,2017-03-12 18:09:51,2017-03-12 18:34:34,153,E 40 St & 5 Ave,40.752062,-73.981632,...,E 3 St & 1 Ave,40.724677,-73.987834,17325,Subscriber,1969.0,2,40.724677,-73.987834,1


In [31]:
beetwen_groups = station_label.loc[station_label.label_x != station_label.label_y,:].groupby(
    ['label_x','label_y']).size().reset_index().rename(columns = {0:'count'}).sort_values(
    'count',ascending=False).reset_index(drop = True)

In [32]:
res = []
for k,v in border_cluster.items():
    res.append(beetwen_groups.loc[(beetwen_groups.label_x == k) & [i not in v for i in beetwen_groups.label_y],
                                 :])
res2 = []
for i in res:
    res2.append(i['count'].sum())

## czyli tylko 7.4% jazd konczy sie gdzieś dalej...

In [34]:
sum(res2)/station_label.shape[0]

0.07421880347228876

## Polowa konczy sie w tym samym "sektorze"

In [36]:
station_label.loc[station_label.label_x != station_label.label_y,:].shape

(8859122, 21)

In [37]:
station_label.shape

(16363616, 21)